In [ ]:
#import packages
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Define the transform to convert the images to PyTorch tensors and normalize the pixel values
trsform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))]) #transforms.Normalize : normalize the color range to [0,1]

In [ ]:
# training dataset and test dataset
train_set = torchvision.datasets.EMNIST(root='./data', split='balanced', train=True, download=True, transform=trsform)
test_set = torchvision.datasets.EMNIST(root='./data', split='balanced', train=False,download=True, transform=trsform)

  0%|          | 0/561753746 [00:00<?, ?it/s]

Extracting ./data/EMNIST/raw/gzip.zip to ./data/EMNIST/raw


In [ ]:
# Define the data loaders to load the data in batches
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_set, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_set, batch_size=64, shuffle=False)

In [ ]:
import torch.nn as nn

# build neural network
class NeuralNetwork(torch.nn.Module):
    def __init__(self, dropout_rate):
        super().__init__()
        self.hidden = torch.nn.Linear(28*28, 256) # input layer 784 neurons , hidden layer with 256 neurons
        self.bnormal1 = nn.BatchNorm1d(256)
        self.sigmoid = torch.nn.Sigmoid() 
        self.hidden1 = torch.nn.Linear(256, 64) #hidden layer with 256 neurons
        self.relu = torch.nn.ReLU()
        self.bnormal2 = nn.BatchNorm1d(64)
        self.output = torch.nn.Linear(64, 47)  #output layer
        self.dropout = nn.Dropout(dropout_rate) # add dropout layer with specified dropout rate
      

    def forward(self, x):
        x = x.to(torch.float32)
        x = x.view(-1, 28*28)
        x = self.hidden(x) 
        x = self.bnormal1(x)
        x = self.sigmoid(x)
        x = self.dropout(x) #dropout for the first hidden layer
        x = self.hidden1(x)
        x = self.relu(x)
        x = self.bnormal2(x)
        x = self.dropout(x) #dropout for the second hidden layer
        x = self.output(x)
        return x


In [ ]:
dropout_rate = 0.1 # specify the dropout rate
net = NeuralNetwork(dropout_rate) # instantiate the MLP neural network
net

NeuralNetwork(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (bnormal1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (sigmoid): Sigmoid()
  (hidden1): Linear(in_features=256, out_features=64, bias=True)
  (relu): ReLU()
  (bnormal2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (output): Linear(in_features=64, out_features=47, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss() # Define the loss function
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) # Define the optimizer

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda:0


In [ ]:
# if we can gpu, let use it!
net.to(device)

NeuralNetwork(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (bnormal1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (sigmoid): Sigmoid()
  (hidden1): Linear(in_features=256, out_features=64, bias=True)
  (relu): ReLU()
  (bnormal2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (output): Linear(in_features=64, out_features=47, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [ ]:
from IPython.utils.sysinfo import num_cpus
#train the neural network
num_epochs = 10
for epoch in range(num_epochs):
    for i, data in enumerate(train_dataloader, 0):
    # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        print(f'epoch:{epoch + 1}, batch: {i+1},  loss: {loss.item()}')

Streaming output truncated to the last 5000 lines.
epoch:8, batch: 290,  loss: 1.1411795616149902
epoch:8, batch: 291,  loss: 1.179434061050415
epoch:8, batch: 292,  loss: 1.120952844619751
epoch:8, batch: 293,  loss: 1.2933733463287354
epoch:8, batch: 294,  loss: 1.0598194599151611
epoch:8, batch: 295,  loss: 1.1394129991531372
epoch:8, batch: 296,  loss: 1.432161808013916
epoch:8, batch: 297,  loss: 1.0578584671020508
epoch:8, batch: 298,  loss: 0.8489642143249512
epoch:8, batch: 299,  loss: 0.9986521005630493
epoch:8, batch: 300,  loss: 1.0471547842025757
epoch:8, batch: 301,  loss: 1.023817777633667
epoch:8, batch: 302,  loss: 1.1571261882781982
epoch:8, batch: 303,  loss: 0.8168584704399109
epoch:8, batch: 304,  loss: 0.921758234500885
epoch:8, batch: 305,  loss: 0.8985430002212524
epoch:8, batch: 306,  loss: 1.0149047374725342
epoch:8, batch: 307,  loss: 1.560626745223999
epoch:8, batch: 308,  loss: 1.0742335319519043
epoch:8, batch: 309,  loss: 0.9289897084236145
epoch:8, batch:

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_dataloader:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # calculate outputs by running images through the network
        outputs = net(inputs)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


print(f'Accuracy of the network on the test images: {100 * correct // total} %')

Accuracy of the network on the test images: 69 %


In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for data in train_dataloader:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # calculate outputs by running images through the network
        outputs = net(inputs)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


print(f'Accuracy of the network on the train images: {100 * correct // total} %')

Accuracy of the network on the train images: 70 %
